In [ ]:
import os
import nltk
import numpy as np
import tensorflow as tf
import pandas as pd

def load_dataset(dataset_path_train,word2idx):

    with open(dataset_path_train,'r') as file1:
      
      sentence_one = []
      sentence_two = []
      y_true = []

      max1 = 0
      max2 = 0

      for index, line in enumerate(file1):
        
        if index == 0:
          continue

        s_1 = []
        s_2 = []

        values = line.split("\t")
        
        #Sentence 1
        
        words = nltk.word_tokenize(values[3])
        words = [word.lower() for word in words]

        s_1.extend([word2idx.get(word,word2idx['UNK']) for word in words])

        #Sentence 2
        # words = values[4].split(" ")
        words = nltk.word_tokenize(values[4])
        words = [word.lower() for word in words]

        s_2.extend([word2idx.get(word,word2idx['UNK']) for word in words])
        

        if len(s_1) > max1:
          max1 = len(s_1)

        if len(s_2) > max2:
          max2 = len(s_2)

        y_true.append(np.asarray(values[0]))

        sentence_one.append(np.pad(s_1,(0,41-len(s_1)),'constant',constant_values=(0)))
        sentence_two.append(np.pad(s_2,(0,41-len(s_2)),'constant',constant_values=(0)))


        # self.sentence_one.append(np.asarray(s_1[0:self.sen_len]))
        # self.sentence_two.append(np.asarray(s_2[0:self.sen_len]))

    sentence_one = np.stack(sentence_one)
    sentence_two = np.stack(sentence_two)
    y_true = np.stack(y_true)

    #print self.weights

    print("Max_train:",max1,max2)

    print(sentence_one.shape,sentence_two.shape,y_true.shape)
    
    

    return sentence_one,sentence_two,y_true


def load_dataset_test(dataset_path_test,word2idx):

    with open(dataset_path_test,'r') as file1:
      
      sentence_one_test = []
      sentence_two_test = []
      y_true_test = []

      max1 = 0
      max2 = 0

      for index, line in enumerate(file1):
        
        if index == 0:
          continue

        s_1 = []
        s_2 = []

        values = line.split("\t")
        
        #Sentence 1
        # words = values[3].split(" ")
        words = nltk.word_tokenize(values[3])
        words = [word.lower() for word in words]

        s_1.extend([word2idx.get(word,word2idx['UNK']) for word in words])

        #Sentence 2
        # words = values[4].split(" ")
        words = nltk.word_tokenize(values[4])
        words = [word.lower() for word in words]

        s_2.extend([word2idx.get(word,word2idx['UNK']) for word in words])
        
        if len(s_1) > max1:
          max1 = len(s_1)

        if len(s_2) > max2:
          max2 = len(s_2)


        y_true_test.append(np.asarray(values[0]))

        sentence_one_test.append(np.pad(s_1,(0,41-len(s_1)),'constant',constant_values=(0)))
        sentence_two_test.append(np.pad(s_2,(0,41-len(s_2)),'constant',constant_values=(0)))

        

    print("Max_test:",max1,max2)

    sentence_one_test = np.stack(sentence_one_test)
    sentence_two_test = np.stack(sentence_two_test)
    y_true_test = np.stack(y_true_test)

    
    print(sentence_one_test.shape,sentence_two_test.shape,y_true_test.shape)

    return sentence_one_test,sentence_two_test,y_true_test

In [ ]:
import tensorflow as tf

eps = 1e-6

def cosine_sim(x1,x2):

    # x1 shape : (batch_size, 1, sequence_length, rnn_hidden_size)
    # x2 shape : (batch_size, sequence_length, 1, rnn_hidden_size)

    # Shape : (batch_size, sequence_length, sequence_length)
    numerator = tf.reduce_sum(tf.multiply(x1,x2),axis=-1)

    # Shape : (batch_size, 1, sequence_length)
    x1_norm = tf.sqrt(tf.maximum(tf.reduce_sum(tf.square(x1),axis=-1),eps))

    # Shape : (batch_size, sequence_length, 1)
    x2_norm = tf.sqrt(tf.maximum(tf.reduce_sum(tf.square(x2),axis=-1),eps))

    return numerator / (x1_norm * x2_norm)


def cosine_matrix(x1,x2):

    # x1 shape : (batch_size, sequence_length, rnn_hidden_size)
    # x2 shape : (batch_size, sequence_length, rnn_hidden_size)

    # Shape : (batch_size, 1, sequence_length, rnn_hidden_size)
    x1_exp = tf.expand_dims(x1,1)

    # Shape : (batch_size, sequence_length, 1, rnn_hidden_size)
    x2_exp = tf.expand_dims(x2,2)

    # Shape : (batch_size, sequence_length, sequence_length, rnn_hidden_size)
    cosine_sim_matrix = cosine_sim(x1_exp,x2_exp)

    return cosine_sim_matrix

def get_last_output(output):

    #print tf.shape(output)[1]
    
    output = tf.transpose(output, [1, 0, 2])
    last = tf.gather(output, int(output.get_shape()[0]) - 1)
    
    #last = tf.gather(output, int(tf.shape(output)[0]) - 1)

    # Shape : (batch_size, 1, rnn_hidden_size) 
    return last


def expand_1D(t,weights):

    # Shape : (1, rnn_hidden_size)
    t_exp = tf.expand_dims(t,axis=0)

    # Shape : (perspectives, rnn_hidden_size)
    return tf.multiply(t_exp,weights)

def expand_2D(t,weights):

    # Shape : (sequence_length, 1, rnn_hidden_size)
    t_exp = tf.expand_dims(t,axis=1)

    # Shape : (1, perspectives, rnn_hidden_size)
    weights_exp = tf.expand_dims(weights,axis=0)

    # Shape : (sequence_length, perspectives, rnn_hidden_size)
    return tf.multiply(t_exp,weights_exp)


def full_matching(sentence_a_full,sentence_b_last,weights):

    def single_func(input):

        # Shape : (sequence_length, rnn_hidden_size)
        sentence_a_single = input[0]

        # Shape : (rnn_hidden_size)
        sentence_b_last_single = input[1]

        # Shape : (sequence_length, perspectives, rnn_hidden_size)
        sentence_a_single = expand_2D(sentence_a_single,weights)

        # Shape : (perspectives, rnn_hidden_size)
        sentence_b_last_single = expand_1D(sentence_b_last_single,weights)

        # Shape : (1, perspectives, rnn_hidden_size)
        sentence_b_last_single = tf.expand_dims(sentence_b_last_single,0)

        # Shape : (sequence_length, perspectives)
        return cosine_sim(sentence_a_single,sentence_b_last_single)

    # Shape : (batch_size, sequence_length, perspectives)
    return tf.map_fn(single_func,(sentence_a_full,sentence_b_last),dtype="float")



def pooling_matching(sentence_a_full,sentence_b_full,weights):

    def single_func(input):

        # Shape : (sequence_length, rnn_hidden_size)
        sentence_a_single = input[0]

        # Shape : (sequence_length, rnn_hidden_size)
        sentence_b_single = input[1]

        # Shape : (sequence_length, perspectives, rnn_hidden_size)
        sentence_a_single = expand_2D(sentence_a_single,weights)			
        sentence_b_single = expand_2D(sentence_b_single,weights)

        # Shape : (sequence_length, 1, perspectives, rnn_hidden_size)
        sentence_a_single = tf.expand_dims(sentence_a_single,1)

        # Shape : (1, sequence_length, perspectives, rnn_hidden_size)
        sentence_b_single = tf.expand_dims(sentence_b_single,0)

        # Shape : (sequence_length, sequence_length, perspectives)
        return cosine_sim(sentence_a_single,sentence_b_single)

    # Shape : (batch_size, sequence_length, sequence_length, perspectives)
    match_matrix = tf.map_fn(single_func,(sentence_a_full,sentence_b_full),dtype="float")

    # Shape : (batch_size, sequence_length, perspectives)
    return tf.reduce_max(match_matrix,axis=2)


def weighted_sim(sentence,sim_mat):

    # Shape : (batch_size, sequence_length, sequence_length, 1)
    sim_mat_exp = tf.expand_dims(sim_mat,-1)

    # Shape : (batch_size, 1, sequence_length, rnn_hidden_size)
    sentence = tf.expand_dims(sentence,1)

    # Shape : (batch_size, sequence_length, rnn_hidden_size)
    weighted_sim_mat = tf.reduce_sum(tf.multiply(sentence,sim_mat_exp),axis=2)

    weighted_sim_mat = tf.div(weighted_sim_mat,tf.expand_dims(tf.add(tf.reduce_sum(sim_mat,axis=-1),eps),axis=-1))

    return weighted_sim_mat


def max_sim(sentence_b_full,sim_mat):

    def single_func(input):

        # Shape : (sequence_length, rnn_hidden_size)
        sentence_b_single = input[0]

        # Shape : (sequence_length)
        max_index = input[1]

        # Shape : (sequence_length, rnn_hidden_size)
        return tf.gather(sentence_b_single,max_index)

    max_index = tf.arg_max(sim_mat,2)

    # Shape : (batch_size, sequence_length, rnn_hidden_size)
    return tf.map_fn(single_func,(sentence_b_full,max_index),dtype="float")


def attentive_matching(sentence_a_full,weighted_sim_mat,weights):

    def single_func(input):

        # Shape : (sequence_length, rnn_hidden_size)
        sentence_a_single = input[0]

        # Shape : (sequence_length, rnn_hidden_size)
        sentence_b_single_att = input[1]

        # Shape : (sequence_length, perspectives, rnn_hidden_size)
        sentence_a_single = expand_2D(sentence_a_single,weights)
        sentence_b_single_att = expand_2D(sentence_b_single_att,weights)

        # Shape : (sequence_length, perspectives)
        return cosine_sim(sentence_a_single,sentence_b_single_att)

    # Shape : (batch_size, sequence_length, perspectives)
    return tf.map_fn(single_func,(sentence_a_full,weighted_sim_mat),dtype="float")


def match(sentence_a_fw, sentence_a_bw, sentence_b_fw, sentence_b_bw, perspectives,hidden_size):

    fw_sim_matrix = cosine_matrix(sentence_a_fw,sentence_b_fw)
    bw_sim_matrix = cosine_matrix(sentence_a_bw,sentence_b_bw)

    #sentence_b_len = tf.reduce_sum(sentence_b_fw, 1)



    # tf.get_variable_scope().reuse_variables()
    with tf.variable_scope("Full_matching",reuse=tf.AUTO_REUSE): 


      last_output_b_fw = get_last_output(sentence_b_fw)
      
      weights_fw = tf.get_variable("forward_matching",
        shape=[perspectives,hidden_size],
        dtype="float")

      fw_full_match = full_matching(sentence_a_fw,last_output_b_fw,weights_fw)

      last_output_b_bw = get_last_output(sentence_b_bw)

      weights_bw = tf.get_variable("backward_matching",
        shape=[perspectives,hidden_size],
        dtype="float")
      
      #tf.get_variable_scope().reuse_variables()

      bw_full_match = full_matching(sentence_a_bw,last_output_b_bw,weights_bw)


    with tf.variable_scope("Pooling_matching",reuse=tf.AUTO_REUSE): 

      
      weights_fw = tf.get_variable("forward_matching",
        shape=[perspectives,hidden_size],
        dtype="float")

      fw_pool_match = pooling_matching(sentence_a_fw,sentence_b_fw,weights_fw)

      weights_bw = tf.get_variable("backward_matching",
        shape=[perspectives,hidden_size],
        dtype="float")
      
      #tf.get_variable_scope().reuse_variables()

      bw_pool_match = pooling_matching(sentence_a_bw,sentence_b_bw,weights_bw)

    with tf.variable_scope("Attentive_matching",reuse=tf.AUTO_REUSE): 


      sentence_b_fw_att = weighted_sim(sentence_b_fw,fw_sim_matrix)


      weights_fw = tf.get_variable("forward_matching",
        shape=[perspectives,hidden_size],
        dtype="float")

      fw_att_match = attentive_matching(sentence_a_fw,sentence_b_fw_att,weights_fw)


      sentence_b_bw_att = weighted_sim(sentence_b_bw,bw_sim_matrix)

      weights_bw = tf.get_variable("backward_matching",
        shape=[perspectives,hidden_size],
        dtype="float")
      
      #tf.get_variable_scope().reuse_variables()

      bw_att_match = attentive_matching(sentence_a_bw,sentence_b_bw_att,weights_bw)

    with tf.variable_scope("Max_Attentive_matching",reuse=tf.AUTO_REUSE): 


      sentence_b_fw_att_max = max_sim(sentence_b_fw,fw_sim_matrix)


      weights_fw = tf.get_variable("forward_matching",
        shape=[perspectives,hidden_size],
        dtype="float")

      fw_max_att_match = attentive_matching(sentence_a_fw,sentence_b_fw_att_max,weights_fw)


      sentence_b_bw_att_max = max_sim(sentence_b_bw,bw_sim_matrix)

      weights_bw = tf.get_variable("backward_matching",
        shape=[perspectives,hidden_size],
        dtype="float")
      
      #tf.get_variable_scope().reuse_variables()

      bw_max_att_match = attentive_matching(sentence_a_bw,sentence_b_bw_att_max,weights_bw)

    return [fw_full_match,bw_full_match,fw_pool_match,bw_pool_match,fw_att_match,bw_att_match,fw_max_att_match,bw_max_att_match]


def bidirectional_match(sentence_a_fw,sentence_a_bw,sentence_b_fw,sentence_b_bw,perspectives=20,hidden_size=100):


    match_1_2 = match(sentence_a_fw,sentence_a_bw,sentence_b_fw,sentence_b_bw,perspectives,hidden_size)
    match_2_1 = match(sentence_b_fw,sentence_b_bw,sentence_a_fw,sentence_a_bw,perspectives,hidden_size)

    match_1_2 = tf.concat(match_1_2,2)
    match_2_1 = tf.concat(match_2_1,2)

    return match_1_2,match_2_1


In [ ]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import pandas as pd
import os
import pickle as cPickle
import nltk
nltk.download('punkt')
# from matching import bidirectional_match,get_last_output
# from data_loader import *
tf.reset_default_graph()

def log(message,file_path=os.path.join('/content/drive/MyDrive/ParaphraseDetection/glove_lstm','log.txt')):
  print(message)
  f1=open(file_path, 'a+')
  f1.write(message)
  f1.write("\n")
  f1.close()

def one_hot(batch_size,Y):

  B = np.zeros((batch_size,2))

  B[np.arange(batch_size),Y] = 1

  return B.astype(int)

def length(sequence):
  used = tf.sign(sequence)
  length = tf.reduce_sum(used, 1)
  length = tf.cast(length, tf.int32)
  
  return length

class model:

  def __init__(self):

    self.word2idx = {'PAD' : 0}
    self.weights = []
    
    self.features = {}
    self.word_list = []

    self.glove_path = '/content/drive/MyDrive/ParaphraseDetection/glove.6B.200d.txt'
    
    self.dataset_path_test = '/content/drive/MyDrive/ParaphraseDetection/msr_paraphrase_test.txt'
    self.dataset_path_train = '/content/drive/MyDrive/ParaphraseDetection/msr_paraphrase_train.txt'
    

    self.learning_rate = 0.0001
  
  def load_glove(self):

    glove_cache_file = os.path.join('/content/drive/MyDrive/ParaphraseDetection/cache', 'glove.pkl')
    word2idx_cache_file = os.path.join('/content/drive/MyDrive/ParaphraseDetection/cache', 'word2idx.pkl')
    #if  path already present means glove embediings are already created so just reload it from memory.

    if os.path.isfile(glove_cache_file):
      print('Loading glove embeddings from : ' + glove_cache_file)
      with open(glove_cache_file, 'rb') as f:
        self.weights = cPickle.load(f)
      print('Done')

      print('Loading word2idx from : ' + word2idx_cache_file)
      with open(word2idx_cache_file, 'rb') as f:
        self.word2idx = cPickle.load(f)
      print('Done')

      self.embed_dim = len(self.weights[0])
      self.vocab_size = self.weights.shape[0]
      # self.word2idx['UNK'] = len(self.weights) - 1
      print (self.weights.shape[0])#1,00,003
      print (len(self.weights[0]))#300
      print (len(self.weights))

    else:
            
      print('Creating glove embeddings:')

      with open(self.glove_path,'r') as file:

        for index, line in enumerate(file):
          values = line.split()
          word = values[0]
        
          try:
            word_weights = np.asarray(values[1:],dtype=np.float32)
            if(word_weights.shape[0] == 200):
              self.word2idx[word] = index+1
              self.weights.append(word_weights)

          except ValueError:
            print('Error at line ',index)

          if index == 100000:
            break

      self.embed_dim = len(self.weights[0])
      self.weights.insert(0,np.random.randn(self.embed_dim))

      self.word2idx['UNK'] = len(self.weights)
      self.weights.append(np.random.randn(self.embed_dim))
      
      self.weights = np.stack(self.weights)
      
      self.vocab_size = self.weights.shape[0]

      print('Saving word2idx to: ' + word2idx_cache_file)
      with open(word2idx_cache_file, 'wb') as f:
        cPickle.dump(self.word2idx,f)
      print('Done!')

      print('Saving glove embeddings to: ' + glove_cache_file)
      with open(glove_cache_file, 'wb') as f:
        cPickle.dump(self.weights,f)
      print('Done!')
          

    print(self.vocab_size,self.embed_dim)

    
    print("Shape of glove embeddings:",self.weights.shape)
    # print(self.weights)


if __name__ == '__main__':

  #os.environ['CUDA_VISIBLE_DEVICES'] = ''
  x = model()

  x.sen_len = 41

  x.load_glove()
  

  x.sentence_one,x.sentence_two,x.y_true = load_dataset(x.dataset_path_train,x.word2idx)
  print( x.sentence_one)

  x.sentence_one_test,x.sentence_two_test,x.y_true_test = load_dataset_test(x.dataset_path_test,x.word2idx)
  # exit(0)
  dropout_rate = 0.1
  num_epoch = 20
  batch_size = 256
  print( x.sentence_one_test)

  X_init = tf.placeholder(tf.float32, shape=(x.vocab_size,x.embed_dim))
  print(x.vocab_size)
  print(x.embed_dim)

  sentence_one = tf.placeholder("int32",[batch_size, x.sen_len],name="sentence_one")
  sentence_two = tf.placeholder("int32",[batch_size, x.sen_len],name="sentence_two")

  Y = tf.placeholder("int32",[batch_size, 2],name="true_labels")
  
  embedding_weights = tf.get_variable(name = 'embedding_weights', initializer = X_init, trainable = False)


  # print sentence_one_len

  with tf.name_scope("Word_embeddings"):

    embedded_sentence_one = tf.nn.embedding_lookup(embedding_weights,sentence_one)
    embedded_sentence_two = tf.nn.embedding_lookup(embedding_weights,sentence_two)


  with tf.variable_scope("Context_representation_layer"):

    context_rnn_hidden_size = 100

    sentence_enc_fw = tf.nn.rnn_cell.LSTMCell(context_rnn_hidden_size,state_is_tuple=True)
    sentence_enc_bw = tf.nn.rnn_cell.LSTMCell(context_rnn_hidden_size,state_is_tuple=True)

    # Sentence 1
    # Shape (batch_size, sequence_length, rnn_hidden_size)

    outputs_1, states_1  = tf.nn.bidirectional_dynamic_rnn(
        cell_fw=sentence_enc_fw,
        cell_bw=sentence_enc_bw,
        dtype=tf.float32,
        # sequence_length=length(sentence_one),
        inputs=embedded_sentence_one)

    output_fw_1, output_bw_1 = outputs_1
    states_fw_1, states_bw_1 = states_1

    output_fw_1 = tf.layers.dropout(
      output_fw_1,
      rate=dropout_rate,
      training=False,
      name="sentence_1_fw_dropout")
    
    output_bw_1 = tf.layers.dropout(
      output_bw_1,
      rate=dropout_rate,
      training=False,
      name="sentence_1_bw_dropout")

    tf.get_variable_scope().reuse_variables()

    #Sentence 2

    # Shape (batch_size, sequence_length, rnn_hidden_size)

    outputs_2, states_2  = tf.nn.bidirectional_dynamic_rnn(
        cell_fw=sentence_enc_fw,
        cell_bw=sentence_enc_bw,
        dtype=tf.float32,
        # sequence_length=length(sentence_two),
        inputs=embedded_sentence_two)
    
    output_fw_2, output_bw_2 = outputs_2
    states_fw_2, states_bw_2 = states_2

    output_fw_2 = tf.layers.dropout(
      output_fw_2,
      rate=dropout_rate,
      training=False,
      name="sentence_2_fw_dropout")
    output_bw_2 = tf.layers.dropout(
      output_bw_2,
      rate=dropout_rate,
      training=False,
      name="sentence_2_bw_dropout")


  #tf.get_variable_scope().reuse_variables()
  with tf.variable_scope("Matching_layer"):
    
    # Shape (batch_size, sequence_length, 8*perspectives)


    match_1_2,match_2_1 = bidirectional_match(output_fw_1,output_bw_1,output_fw_2,output_bw_2)

    match_1_2 = tf.layers.dropout(
      match_1_2,
      rate=dropout_rate,
      training=False,
      name="match_1_2_dropout")
    match_2_1 = tf.layers.dropout(
      match_2_1,
      rate=dropout_rate,
      training=False,
      name="match_2_1_dropout")


  ## Aggregation Layer

  with tf.variable_scope("Aggregation_layer"):

    agg_last = []

    # Sentence 1

    with tf.variable_scope("agg_1",reuse=tf.AUTO_REUSE):

      aggregation_enc_fw = tf.nn.rnn_cell.LSTMCell(context_rnn_hidden_size,state_is_tuple=True)
      aggregation_enc_bw = tf.nn.rnn_cell.LSTMCell(context_rnn_hidden_size,state_is_tuple=True)	

      # Shape (batch_size, sequence_length, rnn_hidden_size)


      agg_outputs_1, agg_states_1  = tf.nn.bidirectional_dynamic_rnn(
          cell_fw=aggregation_enc_fw,
          cell_bw=aggregation_enc_bw,
          dtype=tf.float32,
          # sequence_length=length(sentence_one),
          inputs=match_1_2)

      agg_output_fw_1, agg_output_bw_1 = agg_outputs_1

      agg_output_fw_1 = tf.layers.dropout(
        agg_output_fw_1,
        rate=dropout_rate,
        training=False,
        name="agg_1_fw_dropout")
      agg_output_bw_1 = tf.layers.dropout(
        agg_output_bw_1,
        rate=dropout_rate,
        training=False,
        name="agg_1_bw_dropout")

      # Shape : (batch_size, rnn_hidden_size)

      agg_output_fw_1_last = get_last_output(agg_output_fw_1)
      agg_output_bw_1_last = get_last_output(agg_output_bw_1)

      agg_last.append(agg_output_fw_1_last)
      agg_last.append(agg_output_bw_1_last)

    #tf.get_variable_scope().reuse_variables()

    #Sentence 2

    with tf.variable_scope("agg_2",reuse=tf.AUTO_REUSE):


      aggregation_enc_fw_2 = tf.nn.rnn_cell.LSTMCell(context_rnn_hidden_size,state_is_tuple=True)
      aggregation_enc_bw_2 = tf.nn.rnn_cell.LSTMCell(context_rnn_hidden_size,state_is_tuple=True)

      # Shape (batch_size, sequence_length, rnn_hidden_size)

      agg_outputs_2, agg_states_2  = tf.nn.bidirectional_dynamic_rnn(
          cell_fw=aggregation_enc_fw_2,
          cell_bw=aggregation_enc_bw_2,
          dtype=tf.float32,
          # sequence_length=length(sentence_two),
          inputs=match_2_1)
      
      agg_output_fw_2, agg_output_bw_2 = agg_outputs_2

      agg_output_fw_2 = tf.layers.dropout(
        agg_output_fw_2,
        rate=dropout_rate,
        training=False,
        name="agg_2_fw_dropout")
      agg_output_bw_2 = tf.layers.dropout(
        agg_output_bw_2,
        rate=dropout_rate,
        training=False,
        name="agg_2_bw_dropout")


      # Shape : (batch_size, rnn_hidden_size)

      agg_output_fw_2_last = get_last_output(agg_output_fw_2)
      agg_output_bw_2_last = get_last_output(agg_output_bw_2)


      agg_last.append(agg_output_fw_2_last)
      agg_last.append(agg_output_bw_2_last)

    # Shape : (batch_size, 4*rnn_hidden_size)

    combined_agg_last = tf.concat(agg_last,1)


  ##Prediction Layer
  with tf.variable_scope("Prediction_layer",reuse=tf.AUTO_REUSE):

    prediction_layer_1 = tf.layers.dense(combined_agg_last,
      combined_agg_last.get_shape().as_list()[1],
      activation=tf.nn.tanh,
      name="pred_1")
    
    prediction_layer_1 = tf.layers.dropout(
      prediction_layer_1,
      rate=dropout_rate,
      training=False,
      name="pred_dropout")

    prediction_layer_2 = tf.layers.dense(prediction_layer_1,2,name="pred_2")

  with tf.variable_scope("Train_loss_and_acc"):

    y_pred = tf.nn.softmax(prediction_layer_2)
    print(y_pred)
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=prediction_layer_2))
    
    tf.summary.scalar('loss',loss_op)		

    correct_predictions = tf.equal(tf.argmax(y_pred, 1),tf.argmax(Y, 1))

    batch_accuracy = tf.reduce_mean(tf.cast(correct_predictions,"float"))
    tf.summary.scalar('accuracy',batch_accuracy)

    optimizer = tf.train.AdamOptimizer(learning_rate=x.learning_rate)
    train_op = optimizer.minimize(loss_op)

  merged = tf.summary.merge_all()
  saver = tf.train.Saver()


  with tf.Session() as sess:

    trained_model = os.path.join('/content/drive/MyDrive/ParaphraseDetection/glove_lstm', 'model.ckpt')
    trained_model_restore = os.path.join('/content/drive/MyDrive/ParaphraseDetection/glove_lstm', 'model.ckpt.meta')

    if os.path.isfile(trained_model_restore):

      print("Loading saved model...")
      saver.restore(sess, trained_model)

    else:
      train_writer = tf.summary.FileWriter("./loga/1/train",sess.graph)
      print("No saved model found...Training...")
      sess.run(tf.global_variables_initializer(),feed_dict={X_init:x.weights})

      i = 0.0
      sum_acc = 0.0

      for epoch in range(num_epoch):

        for step in range(int(x.sentence_one.shape[0]/batch_size)):

          i += 1

          batch_s1,batch_s2,batch_y = x.sentence_one[step*batch_size:(step+1)*batch_size],\
                        x.sentence_two[step*batch_size:(step+1)*batch_size],\
                        x.y_true[step*batch_size:(step+1)*batch_size]

          #print batch_s1.shape,batch_s2.shape,batch_y.shape

          batch_y = one_hot(batch_size,batch_y.astype(int))

          [summary,_,loss,acc] = sess.run([merged,train_op,loss_op,batch_accuracy],
                  feed_dict={X_init:x.weights,
                  sentence_one: batch_s1,
                  sentence_two: batch_s2,
                  Y:batch_y
                  })

          train_writer.add_summary(summary, i)


          sum_acc += acc

          log("Epoch:" + str(epoch+1) + " Step:" + str(step) + " Loss:" + "{:.4f}".format(loss) + " Batch Acc:" + "{:.4f}".format(acc) + " Mean Batch Acc:" + "{:.4f}".format(sum_acc/i))
    
      save_path = saver.save(sess, trained_model)
      print("Model saved in path: %s" % save_path)

    print("Testing Model...")
    i = 0
    sum_acc = 0.0
    for step in range(int(x.sentence_one_test.shape[0]/batch_size)):

      i += 1

      batch_s1,batch_s2,batch_y = x.sentence_one_test[step*batch_size:(step+1)*batch_size],\
                    x.sentence_two_test[step*batch_size:(step+1)*batch_size],\
                    x.y_true_test[step*batch_size:(step+1)*batch_size]

      batch_y = one_hot(batch_size,batch_y.astype(int))  
  
      [acc] = sess.run([batch_accuracy],
              feed_dict={X_init:x.weights,
              sentence_one: batch_s1,
              sentence_two: batch_s2,
              Y:batch_y
              })
      sum_acc += acc

      log("Batch Test Accuracy:" + "{:.4f}".format(acc) + " Mean Batch Test Acc:" + "{:.4f}".format(sum_acc/i))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Loading glove embeddings from : /content/drive/MyDrive/ParaphraseDetection/cache/glove.pkl
Done
Loading word2idx from : /content/drive/MyDrive/ParaphraseDetection/cache/word2idx.pkl
Done
100003
200
100003
100003 200
Shape of glove embeddings: (100003, 200)
Max_train: 38 41
(4075, 41) (4075, 41) (4075,)
[[ 85049   1179  40720 ...      0      0      0]
 [    40     41    736 ...      0      0      0]
 [   205 100002   1772 ...      0      0      0]
 ...
 [    29     54     34 ...      0      0      0]
 [     1  14675     16 ...      0      0      0]
 [     1   5794   1847 ...      0      0      0]]
Max_test: 36 35
(1724, 41) (1724, 41) (1724,)
[[     1     86     10 ...      0      0      0]
 [   201      5      1 ...      0      0      0]
 [     8   3526   1837 ...      0      0      0]
 ...
 [ 33531   3960   1044 ...      0      0      0]
 [    77    148      1 ...      0     

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:903: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/core.py:268: UserWarning: `tf.layers.dropout` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dropout` instead.
  warnings.warn('`tf.layers.dropout` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/k

Tensor("Train_loss_and_acc/Softmax:0", shape=(256, 2), dtype=float32)
Loading saved model...
INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/ParaphraseDetection/glove_lstm/model.ckpt
Testing Model...
Batch Test Accuracy:0.7266 Mean Batch Test Acc:0.7266
Batch Test Accuracy:0.7500 Mean Batch Test Acc:0.7383
Batch Test Accuracy:0.7188 Mean Batch Test Acc:0.7318
Batch Test Accuracy:0.7109 Mean Batch Test Acc:0.7266
Batch Test Accuracy:0.7227 Mean Batch Test Acc:0.7258
Batch Test Accuracy:0.7070 Mean Batch Test Acc:0.7227
